In [93]:
from cassandra.cluster import Cluster

In [97]:
cluster = Cluster(['localhost'], port=9042) 
session = cluster.connect() 

In [19]:
session.execute('''CREATE KEYSPACE IF NOT EXISTS cbd WITH replication 
                = {'class': 'SimpleStrategy', 'replication_factor': 1}''')
# criar a base de dados

In [196]:
session.execute("DROP TABLE IF EXISTS cbd.inscritos")

In [197]:
session.execute('''
    CREATE TABLE cbd.inscritos (
    nome_estabelecimento VARCHAR,
    distrito VARCHAR,
    natureza_do_estabelecimento VARCHAR,
    tipo_de_ensino VARCHAR,
    curso VARCHAR,
    sexo VARCHAR,
    total INT,
    grande_grupo VARCHAR,
    pais VARCHAR,
    ano INT,
    PRIMARY KEY (ano,nome_estabelecimento,curso,natureza_do_estabelecimento,sexo));
''')

In [9]:
session.execute('''
    CREATE TABLE cbd.diplomados (
    natureza_do_estabelecimento VARCHAR,
    nome_estabelecimento VARCHAR,
    tipo_de_ensino VARCHAR,
    nivel_de_formacao VARCHAR,
    curso VARCHAR,
    distrito VARCHAR,
    sexo VARCHAR,
    total INT,
    grande_grupo VARCHAR,
    pais VARCHAR,
    ano INT,
    PRIMARY KEY (, nome_estabelecimento));
''')

In [176]:
session.execute('TRUNCATE cbd.inscritos')

In [52]:
import glob
import pandas as pd
import os
import re
import uuid

pd.options.mode.chained_assignment = None

# para mudar as colunas de nome
dict_colunas = {'Subsistema de Ensino':'natureza_do_estabelecimento',
                'Estabelecimento':'nome_estabelecimento',
                'Tipo de Ensino':'tipo_de_ensino',
                'Nível de Formação':'nivel_de_formacao',
                'Curso':'curso',
                'Distrito':'distrito',
                'Sexo':'sexo',
                'Total':'total',
                'CNAEF - Grande Grupo':'grande_grupo',
                'País de Nacionalidade':'pais',
                'N.º de alunos inscritos':'total',
                'N':'total'}

# path onde estao os ficheiros excel
path = r'C:\Users\bruno\Desktop\Projeto CBD\Dados\Inscritos'

excel_files = glob.glob(path + '/*.xlsx')

pattern = r'\d+'


for idx, file in enumerate(excel_files, 1):
    # Create an ExcelFile object without reading the file into memory
    print('Dataset:',idx)
    df = pd.read_excel(file)
    df.rename(columns=dict_colunas, inplace=True)
    # ficar so com as colunas que o df tem no dicionario
    df_filtered = df[[col for col in df.columns if col in dict_colunas.values()]]
    # tirar o ano do ficheiro
    match = re.search(pattern, file)
    ano = int(match.group())
    df_filtered['ano'] = ano
    # as ultimas 2 linhas, 1 delas é vazia e a outra tem o total de tudo
    df_filtered = df_filtered[:-2]
    # para conseguir colocar NaNs no cassandra
    df_filtered.replace({pd.NA: None}, inplace=True)
    # colunas para o insert
    columns = list(df_filtered.columns)
    columns_insert=','.join(columns) 
    # total para int
    df_filtered['total'] = df_filtered['total'].astype(int)
    for _, row in df_filtered.iterrows():
        # colocar '' nas strings
        row_values = [f"'{val}'" if isinstance(val, str) else str(val) for val in row.values]
    
        query = f'''INSERT INTO cbd.inscritos ({columns_insert}) VALUES ({','.join(row_values)});'''
        
        session.execute(query)

    
    


Dataset: 1
Dataset: 2


KeyboardInterrupt: 

In [76]:
import glob
import pandas as pd
import os
import re
import uuid

pd.options.mode.chained_assignment = None

# para mudar as colunas de nome
dict_colunas = {'Subsistema de Ensino':'natureza_do_estabelecimento',
                'Estabelecimento':'nome_estabelecimento',
                'Tipo de Ensino':'tipo_de_ensino',
                'Nível de Formação':'nivel_de_formacao',
                'Curso':'curso',
                'Distrito':'distrito',
                'Sexo':'sexo',
                'Total':'total',
                'CNAEF - Grande Grupo':'grande_grupo',
                'País de Nacionalidade':'pais',
                'N.º de alunos inscritos':'total',
                'N':'total'}

# path onde estao os ficheiros excel
path = r'C:\Users\bruno\Desktop\Projeto CBD\Dados\Diplomados'

excel_files = glob.glob(path + '/*.xlsx')

pattern = r'\d+'


for idx, file in enumerate(excel_files, 1):
    # Create an ExcelFile object without reading the file into memory
    print('Dataset:',idx)
    df = pd.read_excel(file)
    df.rename(columns=dict_colunas, inplace=True)
    # ficar so com as colunas que o df tem no dicionario
    df_filtered = df[[col for col in df.columns if col in dict_colunas.values()]]
    # tirar o ano do ficheiro
    match = re.search(pattern, file)
    ano = int(match.group())
    df_filtered['ano'] = ano
    # as ultimas 2 linhas, 1 delas é vazia e a outra tem o total de tudo
    df_filtered = df_filtered[:-2]
    # para conseguir colocar NaNs no cassandra
    df_filtered.replace({pd.NA: None}, inplace=True)
    # colunas para o insert
    columns = list(df_filtered.columns)
    columns_insert=','.join(columns) 
    # total para int
    df_filtered['total'] = df_filtered['total'].astype(int)
    print(len(df),len(df_filtered))
    for _, row in df_filtered.iterrows():
        # colocar '' nas strings
        row_values = [f"'{val}'" if isinstance(val, str) else str(val) for val in row.values]
    
        query = f'''INSERT INTO cbd.diplomados ({columns_insert}) VALUES ({','.join(row_values)});'''
        
        session.execute(query)

    
    


Dataset: 1
6446 6444
Dataset: 2
6101 6099
Dataset: 3
8066 8064


KeyboardInterrupt: 

In [58]:
session.execute('CREATE INDEX ON cbd.inscritos (distrito);')

In [71]:
# Execute the query
rows = session.execute('''
    SELECT distrito, SUM(total) AS n
    FROM cbd.inscritos
    WHERE ano = 2004 
    GROUP BY distrito
    ALLOW FILTERING;
''')
rows2 = session.execute('''
    SELECT distrito, SUM(total) AS n
    FROM cbd.diplomados
    WHERE ano = 2004 
    GROUP BY distrito
    ALLOW FILTERING;
''')
# Convert the rows to a list and sort them based on total_enrollment
sorted_rows = sorted(rows, key=lambda x: x.n, reverse=True)
sorted_rows2 = sorted(rows2, key=lambda x: x.n, reverse=True)
# Print the sorted rows
for row in sorted_rows2:
    print(row.distrito, row.n)


Coimbra 140
Beja 56
Porto 54
Ilha de São Miguel 46
Vila Real 43
Ilha Terceira 43
Bragança 31
Portalegre 27
Braga 25
Évora 25
Viseu 20
Santarém 11
Guarda 9
Setúbal 7
Ilha da Madeira 6
Aveiro 4
Faro 3
Castelo Branco 2
Viana do Castelo 1
Lisboa 0
Leiria 0


In [78]:
import subprocess

file_path = r"C:\Users\bruno\Desktop\Projeto CBD\Projeto CBD git\teste.csv"
dict_colunas = {'Subsistema de Ensino':'natureza_do_estabelecimento',
                'Estabelecimento':'nome_estabelecimento',
                'Tipo de Ensino':'tipo_de_ensino',
                'Nível de Formação':'nivel_de_formacao',
                'Curso':'curso',
                'Distrito':'distrito',
                'Sexo':'sexo',
                'Total':'total',
                'CNAEF - Grande Grupo':'grande_grupo'}
#session.execute(f"COPY keyspace.diplomados (distrito, nome_estabelecimento, tipo_de_ensino, natureza_do_estabelecimento,nivel_de_formacao,curso,sexo,grande_grupo,pais,total) FROM {file_path} WITH DELIMITER=',' AND HEADER=TRUE AND COLUMN_MAPPING ={dict_colunas};")
file_path = r"C:\Users\bruno\Desktop\Projeto CBD\Projeto CBD git\teste.csv"
keyspace_name = "cbd"
table_name = "diplomados"

column_mapping_string = ','.join([f"{csv_col}:{table_col}" for csv_col, table_col in dict_colunas.items()])

# Construct the cqlsh command to execute
command = f"cqlsh -e \"COPY {keyspace_name}.{table_name} FROM '{file_path}' WITH DELIMITER=',' AND HEADER=TRUE AND COLUMN_MAPPING = {{{column_mapping_string}}};\""

# Execute the cqlsh command
subprocess.run(command, shell=True)

CompletedProcess(args='cqlsh -e "COPY cbd.diplomados FROM \'C:\\Users\\bruno\\Desktop\\Projeto CBD\\Projeto CBD git\\teste.csv\' WITH DELIMITER=\',\' AND HEADER=TRUE AND COLUMN_MAPPING = {Subsistema de Ensino:natureza_do_estabelecimento,Estabelecimento:nome_estabelecimento,Tipo de Ensino:tipo_de_ensino,Nível de Formação:nivel_de_formacao,Curso:curso,Distrito:distrito,Sexo:sexo,Total:total,CNAEF - Grande Grupo:grande_grupo};"', returncode=1)

In [36]:
import subprocess

# Define the command as a string
command = 'cqlsh -e "COPY cbd.diplomados FROM \'C:\\Users\\bruno\\Desktop\\Projeto CBD\\Projeto CBD git\\teste.csv\' WITH DELIMITER=\';\' AND HEADER=TRUE AND COLUMN_MAPPING = {Subsistema de Ensino:natureza_do_estabelecimento,Estabelecimento:nome_estabelecimento,Tipo de Ensino:tipo_de_ensino,Nível de Formação:nivel_de_formacao,Curso:curso,Distrito:distrito,Sexo:sexo,Total:total,CNAEF - Grande Grupo:grande_grupo};"'

# Run the command and capture the error output
process = subprocess.Popen(command, shell=True, stderr=subprocess.PIPE)
out, err = process.communicate()

# Check if there was an error
if err:
    # Print the error output
    print("Error:", err.decode('utf-8'))
else:
    print("Command executed successfully")


Error: 'cqlsh' is not recognized as an internal or external command,
operable program or batch file.



In [39]:
import docker

# Connect to the Docker daemon
client = docker.from_env()

# Find the Cassandra container
cassandra_container = client.containers.get('cassandra')

# Execute a command inside the Cassandra container
command = 'select * from cbd.isncritos'
exec_result = cassandra_container.exec_run(command, tty=True)

# Check the result
if exec_result.exit_code == 0:
    print("Data import successful!")
else:
    print("Data import failed.")


Data import failed.


In [79]:
import pandas as pd

# Load the Excel file into a DataFrame
file = r"C:\Users\bruno\Desktop\Projeto CBD\Dados\Diplomados\Diplomados_2004_2005.xlsx"
f = pd.read_excel(file)
df.rename(columns=dict_colunas, inplace=True)
# ficar so com as colunas que o df tem no dicionario
df_filtered = df[[col for col in df.columns if col in dict_colunas.values()]]
# tirar o ano do ficheiro
match = re.search(pattern, file)
ano = int(match.group())
df_filtered['ano'] = ano
# as ultimas 2 linhas, 1 delas é vazia e a outra tem o total de tudo
df_filtered = df_filtered[:-2]
# para conseguir colocar NaNs no cassandra
df_filtered.replace({pd.NA: None}, inplace=True)
# colunas para o insert
columns = list(df_filtered.columns)
columns_insert=','.join(columns) 
# total para int
df_filtered['total'] = df_filtered['total'].astype(int)
# Write the DataFrame to a CSV file
csv_file = "teste.csv"
df_filtered.to_csv(csv_file, index=False,encoding='utf-8')


In [91]:
import docker

# Conecte-se ao daemon do Docker
client = docker.from_env()

# Encontre o contêiner do Cassandra
cassandra_container = client.containers.get('cassandra')

# Copie o arquivo CSV para dentro do contêiner
csv_file_path = r'C:\Users\bruno\Desktop\Projeto CBD\Projeto CBD git\teste.csv'
container_csv_path = '/teste.csv'

# Execute o comando COPY dentro do contêiner
command = f"copy cbd.inscritos (nome_estabelecimento,distrito,natureza_do_estabelecimento,tipo_de_ensino,curso,sexo,total,ano) from 'teste.csv' with header = true;"
exec_result = cassandra_container.exec_run(command, tty=True)

# Verifique o resultado
if exec_result.exit_code == 0:
    print("Data import successful!")
else:
    print("Data import failed.")


Data import failed.


In [83]:
import docker

# Conecte-se ao daemon do Docker
client = docker.from_env()

# Encontre o contêiner do Cassandra
cassandra_container = client.containers.get('cassandra')

# Execute um comando dentro do contêiner para listar o diretório atual
exec_result = cassandra_container.exec_run(['ls', '-l', '/'])

# Imprima o resultado
print(exec_result.output.decode('utf-8'))

total 68
lrwxrwxrwx   1 root root    7 Feb 27 15:59 bin -> usr/bin
drwxr-xr-x   2 root root 4096 Apr 18  2022 boot
-rwxrwxr-x   1 root root 1182 Mar 28 01:55 __cacert_entrypoint.sh
drwxr-xr-x   5 root root  340 Apr 14 16:57 dev
drwxr-xr-x   1 root root 4096 Apr  8 20:19 etc
drwxr-xr-x   2 root root 4096 Apr 18  2022 home
-rw-r--r--   1 root root    0 Apr 14 20:42 import_cbd_inscritos.err
-rw-r--r--   1 root root    0 Apr 14 20:40 import_cbd_inscritos.err.20240414_204255
lrwxrwxrwx   1 root root    7 Feb 27 15:59 lib -> usr/lib
lrwxrwxrwx   1 root root    9 Feb 27 15:59 lib32 -> usr/lib32
lrwxrwxrwx   1 root root    9 Feb 27 15:59 lib64 -> usr/lib64
lrwxrwxrwx   1 root root   10 Feb 27 15:59 libx32 -> usr/libx32
drwxr-xr-x   2 root root 4096 Feb 27 15:59 media
drwxr-xr-x   2 root root 4096 Feb 27 15:59 mnt
drwxr-xr-x   1 root root 4096 Mar 28 02:49 opt
dr-xr-xr-x 269 root root    0 Apr 14 16:57 proc
drwx------   1 root root 4096 Apr 12 00:16 root
drwxr-xr-x   5 root root 4096 Feb 27 16:

In [125]:
import subprocess


# Define the command to execute
command = r'docker cp "C:\Users\bruno\Desktop\Projeto CBD\Projeto CBD git\modified_teste.csv" cassandra:/'

# Execute the command in a Bash terminal
subprocess.run(command, shell=True)

CompletedProcess(args='docker cp "C:\\Users\\bruno\\Desktop\\Projeto CBD\\Projeto CBD git\\modified_teste.csv" cassandra:/', returncode=0)

In [101]:
dict_colunas = {'Subsistema de Ensino':'natureza_do_estabelecimento',
                'Estabelecimento':'nome_estabelecimento',
                'Tipo de Ensino':'tipo_de_ensino',
                'Nível de Formação':'nivel_de_formacao',
                'Curso':'curso',
                'Distrito':'distrito',
                'Sexo':'sexo',
                'Total':'total',
                'CNAEF - Grande Grupo':'grande_grupo',
                'País de Nacionalidade':'pais',
                'N.º de alunos inscritos':'total',
                'N':'total'}

In [123]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('teste.csv')

missing_columns = ['nivel_de_formacao', 'grande_grupo', 'pais']

# Change data type of specified columns to string
df[missing_columns] = ''
# Reorder columns to match the table schema
df = df[['ano','distrito','nome_estabelecimento','curso','grande_grupo','natureza_do_estabelecimento','nivel_de_formacao','pais','sexo','tipo_de_ensino','total']]

# Save the modified DataFrame back to a CSV file
df.to_csv('modified_teste.csv', index=False)



In [104]:
df = pd.read_csv('modified_teste.csv')
column_types = df.dtypes
print(column_types)

nome_estabelecimento            object
distrito                        object
natureza_do_estabelecimento     object
tipo_de_ensino                  object
curso                           object
sexo                            object
total                            int64
ano                              int64
nivel_de_formacao              float64
grande_grupo                   float64
pais                           float64
dtype: object


In [ ]:
import pandas as pd

# Create a DataFrame with a column containing None values


# Check the data types of the columns before changing
print("Before changing:")
print(df.dtypes)

# Change the data type of the column to integer
df['col1'] = df['col1'].astype('int')

# Check the data types of the columns after changing
print("\nAfter changing:")
print(df.dtypes)


In [181]:
import glob
import pandas as pd
import re
import subprocess

pd.options.mode.chained_assignment = None

# Dictionary for column renaming
dict_colunas = {
    'Subsistema de Ensino': 'natureza_do_estabelecimento',
    'Estabelecimento': 'nome_estabelecimento',
    'Tipo de Ensino': 'tipo_de_ensino',
    'Curso': 'curso',
    'Distrito': 'distrito',
    'Sexo': 'sexo',
    'Total': 'total',
    'CNAEF - Grande Grupo': 'grande_grupo',
    'País de Nacionalidade': 'pais',
    'N.º de alunos inscritos': 'total',
    'N': 'total',
    'CNAEF - Grande grupo' : 'grande_grupo',
    'Área de Educação e Formação - Área Específica':'grande_grupo',
    'Natureza do Estabelecimento de Ensino':'natureza_do_estabelecimento'

}

# Path to the directory containing Excel files
path = r'C:\Users\bruno\Desktop\Projeto CBD\Dados\Inscritos'

excel_files = glob.glob(path + '/*.xlsx')

pattern = r'\d+'

combined_df = pd.DataFrame(columns=['ano', 'distrito', 'nome_estabelecimento', 'curso', 'grande_grupo', 'natureza_do_estabelecimento',
                                    'pais', 'sexo', 'tipo_de_ensino', 'total'])

for idx, file in enumerate(excel_files, 1):
    print('Dataset:', idx)
    # Read Excel file
    df = pd.read_excel(file)
    
    # Rename columns
    df.rename(columns=dict_colunas, inplace=True)
    
    # Check and add missing columns
    for col in dict_colunas.values():
        if col not in df.columns:
            df[col] = ''
    
    # Extract year from filename
    match = re.search(pattern, file)
    ano = int(match.group())
    
    # Replace NaNs with None
    df.replace({pd.NA: None}, inplace=True)
    column_to_check = 'nome_estabelecimento'
    # Remove rows where the specified column has an empty value
    df = df[df[column_to_check].notna()]

    # Convert 'total' column to integer
    df['total'] = df['total'].astype(int)
    
    # Add 'ano' column
    df['ano'] = ano
    
    # Select specific columns
    df = df[['ano', 'distrito', 'nome_estabelecimento', 'curso', 'grande_grupo', 'natureza_do_estabelecimento',
             'pais', 'sexo', 'tipo_de_ensino', 'total']]
    
    # Save to CSV
    combined_df = pd.concat([combined_df, df], ignore_index=True)

combined_df.to_csv('CR7.csv', index=False,encoding='utf-8')




Dataset: 1
Dataset: 2
Dataset: 3
Dataset: 4
Dataset: 5
Dataset: 6
Dataset: 7
Dataset: 8
Dataset: 9
Dataset: 10
Dataset: 11
Dataset: 12
Dataset: 13
Dataset: 14
Dataset: 15
Dataset: 16
Dataset: 17
Dataset: 18


In [ ]:
command = r'docker cp "C:\Users\bruno\Desktop\Projeto CBD\Projeto CBD git\CR7.csv" cassandra:/'

# Execute the command in a Bash terminal
subprocess.run(command, shell=True)

In [198]:
command = r"COPY cbd.inscritos  (ano,distrito,nome_estabelecimento,curso,grande_grupo,natureza_do_estabelecimento,pais,sexo,tipo_de_ensino,total)   FROM 'CR7.csv'   WITH HEADER = true;"

subprocess.run(['docker', 'exec', '-i', 'cassandra', 'cqlsh', '-e', command], check=True)

CompletedProcess(args=['docker', 'exec', '-i', 'cassandra', 'cqlsh', '-e', "COPY cbd.inscritos  (ano,distrito,nome_estabelecimento,curso,grande_grupo,natureza_do_estabelecimento,pais,sexo,tipo_de_ensino,total)   FROM 'CR7.csv'   WITH HEADER = true;"], returncode=0)

In [175]:
command = r'''COPY cbd.inscritos 
(ano,distrito,nome_estabelecimento,curso,grande_grupo,natureza_do_estabelecimento,nivel_de_formacao,pais,sexo,tipo_de_ensino,total)  
FROM 'CR7.csv'  
WITH HEADER = true;'''

subprocess.run(command, shell=True)

CompletedProcess(args="COPY cbd.inscritos \n(ano,distrito,nome_estabelecimento,curso,grande_grupo,natureza_do_estabelecimento,nivel_de_formacao,pais,sexo,tipo_de_ensino,total)  \nFROM 'CR7.csv'  \nWITH HEADER = true;", returncode=1)

In [ ]:
ano int,
    distrito text,
    nome_estabelecimento text,
    curso text,
    grande_grupo text,
    natureza_do_estabelecimento text,
    nivel_de_formacao text,
    pais text,
    sexo text,
    tipo_de_ensino text,
    total int,

In [151]:
import pandas as pd

# Provide the full path to the Excel file
file_path = r"C:\Users\bruno\Desktop\Projeto CBD\Dados\Inscritos\Inscritos_2012_2013.xlsx"

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Save the DataFrame as a CSV file
df.to_csv('t.csv', index=False)
